In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# RQ1: Which technologies have been investigated in the last decade?

In [3]:
#Datene einlesen
data = pd.read_csv('./data/data.csv')
data.drop(["Unnamed: 0"], axis = 1, inplace = True)
data.head()

,Document Title,Authors,Author Affiliations,Publication Title,Date Added To Xplore,Publication Year,Volume,Issue,Start Page,End Page,...,Mesh_Terms,Article Citation Count,Patent Citation Count,Reference Count,License,Online Date,Issue Date,Meeting Date,Publisher,Document Identifier
0,What Do We Know about Knowledge Management? Pr...,T. Dingsøyr; F. O. Bjørnson; F. Shull,SINTEF Information and Communication Technolog...,IEEE Software,17 Apr 2009,2009,26.0,3.0,100,103,...,NaN,28.0,NaN,7.0,NaN,17 Apr 2009,NaN,NaN,IEEE,IEEE Magazines
1,Research on National and International Softwar...,G. Li; H. Dong; Q. Zheng; M. Zhou; Y. Guo,"Shandong Comput. Sci. Center, Jinan, China; Sh...",2009 International Conference on Computational...,28 Dec 2009,2009,NaN,NaN,1,4,...,NaN,NaN,NaN,7.0,NaN,28 Dec 2009,NaN,NaN,IEEE,IEEE Conferences
2,Advancing Software Engineering Professional Ed...,M. Ardis; P. Bourque; T. Hilburn; K. Lasfer; S...,Stevens Institute of Technology; École de Tech...,IEEE Software,23 Jun 2011,2011,28.0,4.0,58,63,...,NaN,11.0,NaN,15.0,NaN,30 Sep 2010,NaN,NaN,IEEE,IEEE Magazines
3,Putting Human Aspects of Software Engineering ...,O. Hazzan,Technion–Israel Institute of Technology,IEEE Software,14 Jun 2010,2010,27.0,4.0,90,91,...,NaN,3.0,NaN,7.0,NaN,14 Jun 2010,NaN,NaN,IEEE,IEEE Magazines
4,Where's the Theory for Software Engineering?,P. Johnson; M. Ekstedt; I. Jacobson,KTH Royal Institute of Technology; KTH Royal I...,IEEE Software,21 Aug 2012,2012,29.0,5.0,96,96,...,NaN,17.0,NaN,NaN,NaN,21 Aug 2012,NaN,NaN,IEEE,IEEE Magazines


In [4]:
data.columns

Index(['Document Title', 'Authors', 'Author Affiliations', 'Publication Title',
       'Date Added To Xplore', 'Publication Year', 'Volume', 'Issue',
       'Start Page', 'End Page', 'Abstract', 'ISSN', 'ISBNs', 'DOI',
       'Funding Information', 'PDF Link', 'Author Keywords', 'IEEE Terms',
       'INSPEC Controlled Terms', 'INSPEC Non-Controlled Terms', 'Mesh_Terms',
       'Article Citation Count', 'Patent Citation Count', 'Reference Count',
       'License', 'Online Date', 'Issue Date', 'Meeting Date', 'Publisher',
       'Document Identifier'],
      dtype='object')

In [5]:
data.count()

Document Title                 3592
Authors                        3408
Author Affiliations            3347
Publication Title              3592
Date Added To Xplore           3295
Publication Year               3592
Volume                         2298
Issue                          2298
Start Page                     3592
End Page                       3589
Abstract                       3576
ISSN                           2298
ISBNs                          1294
DOI                            3592
Funding Information             538
PDF Link                       3592
Author Keywords                1982
IEEE Terms                     3271
INSPEC Controlled Terms        2783
INSPEC Non-Controlled Terms    2783
Mesh_Terms                        0
Article Citation Count         2101
Patent Citation Count           108
Reference Count                2936
License                         286
Online Date                    3592
Issue Date                        0
Meeting Date                

In [6]:
def CreateTermDocMatrix(column):
        
    count_vectorizer = CountVectorizer(stop_words='english',ngram_range = (1,2))
    sparse_matrix = count_vectorizer.fit_transform(data[column].values.astype('U'))

    doc_term_matrix = sparse_matrix.todense()
    df = pd.DataFrame(doc_term_matrix, columns=count_vectorizer.get_feature_names())
    df['PublishingYear'] = data['Publication Year']
    returndf = df.groupby('PublishingYear').sum().transpose()
    return returndf

In [7]:
def ShowWord(data,word):
    d = data.loc[word]
    d.plot.bar()

In [8]:
def SpaltenEntf(data):
    y = []
    for item in data.columns:
        y = data.columns


    for index in range(len(y)):
        x = y[index]
        if x in stopwords or x.isnumeric():
            del data[y[index]]
    return data

In [9]:
#Untersuchung Abstract
abstract = CreateTermDocMatrix('Abstract')

MemoryError: Unable to allocate 5.47 GiB for an array with shape (204412, 3592) and data type int64

In [ ]:
b = abstract.loc[abstract[2011] > 20] 

In [ ]:
b.plot.bar()
print(b)

In [ ]:
import pandas as pd
from bokeh.plotting import ColumnDataSource
from bokeh.plotting import figure
from bokeh.plotting import show

In [ ]:
plot = figure(title="Anzahl der Infizierten in Kiel (März 2020 - März 2021)", width=800, x_axis_label="Datum", x_axis_type="datetime", y_axis_label="Infizierte")

In [ ]:
datenquelle = ColumnDataSource(b)

In [ ]:
plot.line(source=datenquelle, x="Datum", y="Infizierte aktuell")

In [ ]:
#Untersuchung Title
#title = CreateTermDocMatrix('Title')

#print(title)
#title = SpaltenEntf(title)
#print(title)

#b = title.loc[title[2017] > 5] 

#b.plot.bar()
#print(b)
#b.shape

In [ ]:
#Einzelnes Wort anzeigen
#abstract = CreateTermDocMatrix('Abstract')
#ShowWord(abstract,'software')